# Generating operating data using simulation

We can use our model to generate a wide range of operation data to:
- learn about the product behavior
- anticipate how uncertainties will affect our understanding
- have data to feed AI algos
- etc.

### Model setup

In [ ]:
from cpu.systems import CPUSystem

cpu = CPUSystem.load("data/cpu_ref.json")

### Dysfunctional case definition

The model can represent a [nominal/fully working product](generate_nominal_operation_data.ipynb) but also a dysfunctional one!

Here the simulation represents a product with a fan breakage during operation at `t=10s`.

In [ ]:
import numpy as np
import pandas as pd

from cosapp.drivers import NonLinearSolver, EulerExplicit
from cosapp.drivers.time.scenario import Interpolator
from cosapp.recorders import DataFrameRecorder

# for comparison purpose, use the same ambient temp and usage as the nominal case
data = pd.read_csv("data/cpu_hot_day_intensive_use.csv")
mass_flow_scalar = np.concatenate([np.full(10, 1.0), np.full(21, 0.0)])

time_driver = cpu.add_driver(EulerExplicit())
solver = time_driver.add_child(NonLinearSolver("solver", max_iter=10, factor=1.0))

time_driver.time_interval = (0, 30)
time_driver.dt = 1.0

time_driver.set_scenario(
    init={"T_cpu": 10.0, "exchanger.h_adder": -30.0},
    values={
        "fan.mass_flow_scalar": Interpolator(
            np.stack([np.arange(0, 31, 1), mass_flow_scalar], axis=1)
        ),
        "fan.T_air": Interpolator(np.stack([np.arange(0, 31, 1), data["fan.T_air"]], axis=1)),
        "cpu.usage": Interpolator(np.stack([np.arange(0, 31, 1), data["cpu.usage"]], axis=1)),
    },
)

rec = time_driver.add_recorder(DataFrameRecorder(includes=["*"], hold=False), period=1.0)
cpu.run_drivers()

data = rec.data[["fan.T_air", "cpu.usage", "T_cpu", "fan.tension", "time", "fan.mass_flow_scalar"]]

# T_cpu is here generated by simulation, add some noise/uncertainties
data["T_cpu"] += np.random.normal(-0.2, 0.2, data["T_cpu"].shape)

data.to_csv("data/cpu_hot_day_intensive_use_broken.csv")

In [ ]:
from cpu.utils.plot_recorders import plot_recorders

plot_recorders(
    {
        "operation_nominal": pd.read_csv("data/cpu_hot_day_intensive_use.csv"),
        "operation_broken_fan": pd.read_csv("data/cpu_hot_day_intensive_use_broken.csv"),
    },
    [[("time", "fan.T_air"), ("time", "cpu.usage")], [("time", "fan.tension"), ("time", "T_cpu")]],
    width=800,
    height=600,
)